In [1]:
import pandas as pd
import os

event_df = pd.read_csv('../../Our Datasets/processed_recoveries_and_interceptions_dynamic_events.csv')

match_ids = event_df['source_file'].unique()

In [2]:
event_df = event_df.rename(columns={'Unique ID': 'unique_poss_id'})

event_df['source_file'] = event_df['source_file'].astype(str)

This should take 7 minutes roughly to run

In [ ]:
base_dir = "../../data/matches"
all_tracking = []

for mid in match_ids:
    folder = os.path.join(base_dir, str(mid))
    files = os.listdir(folder)

    # find file that contains "tracking_extrapolation"
    track_files = [f for f in files if "tracking_extrapolated" in f.lower()]

    if not track_files:
        print(f"❌ No tracking_extrapolation file for match {mid}")
        continue

    file_path = os.path.join(folder, track_files[0])
    print(f"📄 Reading JSONL: {file_path}")

    # Read JSONL into DataFrame
    tracking_df_temp = pd.read_json(file_path, lines=True)

    tracking_df_temp["match_id"] = mid  # tag match ID
    tracking_df_temp['match_id'] = tracking_df_temp['match_id'].astype(str)
    tracking_df_temp['frame'] = tracking_df_temp['frame'].astype(int)

    merged = tracking_df_temp.merge(
        event_df[['source_file', 'unique_poss_id', 'possession_start', 'possession_end']],
        left_on='match_id',
        right_on='source_file',
        how='left'
    )

    filtered_tracking = merged[
        (merged['frame'] >= merged['possession_start']) &
        (merged['frame'] <= merged['possession_end'])
    ].copy()


    filtered_tracking = pd.json_normalize(
        filtered_tracking.to_dict("records"),
        "player_data",
        ["frame", "timestamp", "period", "possession", "ball_data"],
    )

    # Extract 'player_id' and 'group from the 'possession' dictionary
    filtered_tracking["possession_player_id"] = filtered_tracking["possession"].apply(
        lambda x: x.get("player_id")
    )
    filtered_tracking["possession_group"] = filtered_tracking["possession"].apply(lambda x: x.get("group"))


    # (Optional) Expand the ball_data with json_normalize
    filtered_tracking[["ball_x", "ball_y", "ball_z", "is_detected_ball"]] = pd.json_normalize(
        filtered_tracking.ball_data
    )

    filtered_tracking["match_id"] = mid  # tag match ID
    filtered_tracking['match_id'] = filtered_tracking['match_id'].astype(str)


    all_tracking.append(filtered_tracking
                        
                        )

# combine all matches
tracking_all = pd.concat(all_tracking, ignore_index=True)

tracking_all = tracking_all.drop(columns=["possession", "ball_data"])

📄 Reading JSONL: ../../data/matches\1886347\1886347_tracking_extrapolated.jsonl
📄 Reading JSONL: ../../data/matches\1899585\1899585_tracking_extrapolated.jsonl
📄 Reading JSONL: ../../data/matches\1925299\1925299_tracking_extrapolated.jsonl
📄 Reading JSONL: ../../data/matches\1953632\1953632_tracking_extrapolated.jsonl
📄 Reading JSONL: ../../data/matches\1996435\1996435_tracking_extrapolated.jsonl
📄 Reading JSONL: ../../data/matches\2006229\2006229_tracking_extrapolated.jsonl
📄 Reading JSONL: ../../data/matches\2011166\2011166_tracking_extrapolated.jsonl
📄 Reading JSONL: ../../data/matches\2013725\2013725_tracking_extrapolated.jsonl
📄 Reading JSONL: ../../data/matches\2015213\2015213_tracking_extrapolated.jsonl
📄 Reading JSONL: ../../data/matches\2017461\2017461_tracking_extrapolated.jsonl


In [4]:
tracking_all

,x,y,player_id,is_detected,frame,timestamp,period,possession_player_id,possession_group,ball_x,ball_y,ball_z,is_detected_ball,match_id
0,-43.88,-7.85,51009,True,283,2025-11-16 00:00:27.300,1.0,NaN,home team,-29.73,-26.19,0.14,True,1886347
1,-28.71,-19.31,176224,True,283,2025-11-16 00:00:27.300,1.0,NaN,home team,-29.73,-26.19,0.14,True,1886347
2,-24.68,-8.52,51649,True,283,2025-11-16 00:00:27.300,1.0,NaN,home team,-29.73,-26.19,0.14,True,1886347
3,-28.54,-24.91,50983,True,283,2025-11-16 00:00:27.300,1.0,NaN,home team,-29.73,-26.19,0.14,True,1886347
4,-24.18,2.01,735578,True,283,2025-11-16 00:00:27.300,1.0,NaN,home team,-29.73,-26.19,0.14,True,1886347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26045795,32.17,11.11,159688,False,69501,2025-11-16 01:34:43.100,2.0,43829.0,away team,37.08,-6.75,0.23,True,2017461
26045796,0.08,17.78,51714,False,69501,2025-11-16 01:34:43.100,2.0,43829.0,away team,37.08,-6.75,0.23,True,2017461
26045797,21.59,11.35,795521,False,69501,2025-11-16 01:34:43.100,2.0,43829.0,away team,37.08,-6.75,0.23,True,2017461
26045798,-7.91,-7.07,745320,False,69501,2025-11-16 01:34:43.100,2.0,43829.0,away team,37.08,-6.75,0.23,True,2017461
